# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [2]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [3]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [4]:
# Your code here
# Step 1: Scale X_train and X_val using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Ensure X_train_scaled and X_val_scaled are DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X.columns)


In [5]:
# Your code here
# Step 2: Create a LinearRegression model and fit it on scaled training data
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Calculate a baseline r-squared score on training data
r2_train = model.score(X_train_scaled, y_train)
print(f"Baseline R² score on training data: {r2_train:.4f}")


Baseline R² score on training data: 0.7868


## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [6]:
# Your code here
# Step 1: Set up data structure to store interaction pairs and their R² scores
interaction_results = []

# Step 2: Find combinations of columns and loop over them
for col1, col2 in combinations(X.columns, 2):
    # Make copies of X_train and X_val
    X_train_interaction = X_train.copy()
    X_val_interaction = X_val.copy()
    
    # Add interaction term to data
    X_train_interaction[f'{col1}_x_{col2}'] = X_train_interaction[col1] * X_train_interaction[col2]
    X_val_interaction[f'{col1}_x_{col2}'] = X_val_interaction[col1] * X_val_interaction[col2]
    
    # Scale the new features
    X_train_interaction_scaled = scaler.fit_transform(X_train_interaction)
    X_val_interaction_scaled = scaler.transform(X_val_interaction)
    
    # Fit the model on the new training data
    model.fit(X_train_interaction_scaled, y_train)
    
    # Find r-squared score (fit on training data, evaluate on validation data)
    r2_score = model.score(X_val_interaction_scaled, y_val)
    
    # Append to data structure
    interaction_results.append((col1, col2, r2_score))

# Step 3: Sort and subset the data structure to find the top 7
interaction_results.sort(key=lambda x: x[2], reverse=True)
top_interactions = interaction_results[:7]

# Print the top 7 interactions and their R² scores
for interaction in top_interactions:
    print(f"Interaction: {interaction[0]} * {interaction[1]}, R² Score: {interaction[2]:.4f}")

Interaction: LotArea * 1stFlrSF, R² Score: 0.7211
Interaction: LotArea * TotalBsmtSF, R² Score: 0.7072
Interaction: LotArea * GrLivArea, R² Score: 0.6691
Interaction: LotArea * Fireplaces, R² Score: 0.6530
Interaction: 2ndFlrSF * TotRmsAbvGrd, R² Score: 0.6473
Interaction: OverallCond * TotalBsmtSF, R² Score: 0.6429
Interaction: OverallQual * 2ndFlrSF, R² Score: 0.6422


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [7]:
# Step 1: Loop over top 7 interactions
for interaction in top_interactions:
    # Extract column names from data structure
    col1, col2, _ = interaction  # We don't need the R² score here
    
    # Construct new column name
    new_column_name = f"{col1}_{col2}"
    
    # Add new column to X_train and X_val
    X_train[new_column_name] = X_train[col1] * X_train[col2]
    X_val[new_column_name] = X_val[col1] * X_val[col2]

# Optional: Display the updated X_train and X_val to verify the new columns
print("Updated X_train columns:", X_train.columns.tolist())
print("Updated X_val columns:", X_val.columns.tolist())


Updated X_train columns: ['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageArea', 'Fireplaces', 'LotArea_1stFlrSF', 'LotArea_TotalBsmtSF', 'LotArea_GrLivArea', 'LotArea_Fireplaces', '2ndFlrSF_TotRmsAbvGrd', 'OverallCond_TotalBsmtSF', 'OverallQual_2ndFlrSF']
Updated X_val columns: ['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageArea', 'Fireplaces', 'LotArea_1stFlrSF', 'LotArea_TotalBsmtSF', 'LotArea_GrLivArea', 'LotArea_Fireplaces', '2ndFlrSF_TotRmsAbvGrd', 'OverallCond_TotalBsmtSF', 'OverallQual_2ndFlrSF']


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [10]:
# Your code here
# Step 1: Set up data structure to store polynomial terms and their R² scores
polynomial_results = []

# Check for NaN values in X_train and X_val
if X_train.isnull().values.any() or X_val.isnull().values.any():
    print("Warning: NaN values found in the dataset.")
    # Fill NaN values with the mean of each column
    X_train.fillna(X_train.mean(), inplace=True)
    X_val.fillna(X_val.mean(), inplace=True)

# Check for infinite values in X_train and X_val
if not np.isfinite(X_train.values).all() or not np.isfinite(X_val.values).all():
    print("Warning: Infinite values found in the dataset.")
    # Handle infinite values if necessary (e.g., replace with finite values)
    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_val.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Optionally fill NaN values again after replacing infinite values
    X_train.fillna(X_train.mean(), inplace=True)
    X_val.fillna(X_val.mean(), inplace=True)

# Step 2: Loop over all columns
for col in X.columns:
    # Loop over degrees 2, 3, 4
    for degree in range(2, 5):
        # Make a copy of X_train and X_val
        X_train_poly = X_train.copy()
        X_val_poly = X_val.copy()
        
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        
        # Fit polynomial to column and transform column
        poly_features_train = poly.fit_transform(X_train_poly[[col]])
        poly_features_val = poly.transform(X_val_poly[[col]])
        
        # Convert the result to DataFrame
        poly_features_train_df = pd.DataFrame(poly_features_train, columns=[f"{col}^{d}" for d in range(1, degree + 1)])
        poly_features_val_df = pd.DataFrame(poly_features_val, columns=[f"{col}^{d}" for d in range(1, degree + 1)])
        
        # Add polynomial to data
        # Drop the original column before concatenating
        X_train_poly = pd.concat([X_train_poly.drop(columns=[col]), poly_features_train_df], axis=1)
        X_val_poly = pd.concat([X_val_poly.drop(columns=[col]), poly_features_val_df], axis=1)
        
        # Fit the model on the new training data
        try:
            model.fit(X_train_poly, y_train)
            # Find r-squared score on validation
            r2_score = model.score(X_val_poly, y_val)
            # Append to data structure
            polynomial_results.append((col, degree, r2_score))
        except ValueError as e:
            print(f"Error fitting model for {col}^{degree}: {e}")

# Step 3: Sort and subset the data structure to find the top 7
polynomial_results.sort(key=lambda x: x[2], reverse=True)
top_polynomials = polynomial_results[:7]

# Print the top 7 polynomial terms and their R² scores
for polynomial in top_polynomials:
    print(f"Polynomial: {polynomial[0]}^{polynomial[1]}, R² Score: {polynomial[2]:.4f}")

Error fitting model for LotArea^2: Input contains NaN, infinity or a value too large for dtype('float64').
Error fitting model for LotArea^3: Input contains NaN, infinity or a value too large for dtype('float64').
Error fitting model for LotArea^4: Input contains NaN, infinity or a value too large for dtype('float64').
Error fitting model for OverallQual^2: Input contains NaN, infinity or a value too large for dtype('float64').
Error fitting model for OverallQual^3: Input contains NaN, infinity or a value too large for dtype('float64').
Error fitting model for OverallQual^4: Input contains NaN, infinity or a value too large for dtype('float64').
Error fitting model for OverallCond^2: Input contains NaN, infinity or a value too large for dtype('float64').
Error fitting model for OverallCond^3: Input contains NaN, infinity or a value too large for dtype('float64').
Error fitting model for OverallCond^4: Input contains NaN, infinity or a value too large for dtype('float64').
Error fitting

### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [12]:
# Step 1: Filter out duplicates, keeping only the best polynomial for each column
best_polynomials = {}
for col, degree, r2 in polynomial_results:
    if col not in best_polynomials or r2 > best_polynomials[col][1]:
        best_polynomials[col] = (degree, r2)

# Step 2: Create a list to store new polynomial features
new_polynomial_features_train = []
new_polynomial_features_val = []

# Step 3: Loop over remaining results to create polynomial terms
for col, (degree, r2) in best_polynomials.items():
    # Instantiate PolynomialFeatures with the best degree
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    
    # Create polynomial features for training and validation sets
    poly_features_train = poly.fit_transform(X_train[[col]])
    poly_features_val = poly.transform(X_val[[col]])
    
    # Convert to DataFrame
    poly_features_train_df = pd.DataFrame(poly_features_train, columns=[f"{col}^{d}" for d in range(1, degree + 1)])
    poly_features_val_df = pd.DataFrame(poly_features_val, columns=[f"{col}^{d}" for d in range(1, degree + 1)])
    
    # Append new polynomial features to the list
    new_polynomial_features_train.append(poly_features_train_df)
    new_polynomial_features_val.append(poly_features_val_df)

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [13]:
# Your code here
# Step 4: Concatenate new polynomial features to X_train and X_val
if new_polynomial_features_train:
    X_train_polynomials = pd.concat(new_polynomial_features_train, axis=1)
    X_val_polynomials = pd.concat(new_polynomial_features_val, axis=1)
    
    # Concatenate with original data
    X_train = pd.concat([X_train.reset_index(drop=True), X_train_polynomials.reset_index(drop=True)], axis=1)
    X_val = pd.concat([X_val.reset_index(drop=True), X_val_polynomials.reset_index(drop=True)], axis=1)

# Check the final dataset to ensure polynomial terms are included
print("Final X_train shape:", X_train.shape)
print("Final X_val shape:", X_val.shape)

# Optionally, display the first few rows of the final datasets
print("X_train head:\n", X_train.head())
print("X_val head:\n", X_val.head())

Final X_train shape: (821, 17)
Final X_val shape: (274, 17)
X_train head:
       LotArea  OverallQual  OverallCond  TotalBsmtSF  1stFlrSF  2ndFlrSF  \
518      9531            6            5          794       882       914   
236      8773            7            5         1414      1414         0   
38       7922            5            7         1057      1057         0   
1034     6305            5            7          920       954         0   
520     10800            4            7            0       694       600   

      GrLivArea  TotRmsAbvGrd  GarageArea  Fireplaces  LotArea_1stFlrSF  \
518        1796             7         546           0           8406342   
236        1414             6         494           0          12405022   
38         1057             5         246           0           8373554   
1034        954             5         240           1           6014970   
520        1294             7           0           0           7495200   

      LotArea_Tot

## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [14]:
# Your code here
model.fit(X_train, y_train)

# Step 2: Calculate R² scores for training and validation datasets
train_r2_score = model.score(X_train, y_train)
val_r2_score = model.score(X_val, y_val)

# Step 3: Print the R² scores
print(f"Training R² Score: {train_r2_score:.4f}")
print(f"Validation R² Score: {val_r2_score:.4f}")

Training R² Score: 0.8004
Validation R² Score: 0.7468


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [15]:
# Your code here
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV

# Define the model for RFE
model_rfe = LinearRegression()

# Step 1: RFE with different n_features_to_select values
n_features_options = [5, 10, 15, 20, 25]  # Adjust these values based on your dataset
for n_features in n_features_options:
    rfe = RFE(estimator=model_rfe, n_features_to_select=n_features)
    rfe.fit(X_train, y_train)
    
    # Get selected features
    X_train_rfe = X_train.iloc[:, rfe.support_]
    X_val_rfe = X_val.iloc[:, rfe.support_]
    
    # Fit the model and calculate R² scores
    model_rfe.fit(X_train_rfe, y_train)
    train_r2_score_rfe = model_rfe.score(X_train_rfe, y_train)
    val_r2_score_rfe = model_rfe.score(X_val_rfe, y_val)
    
    print(f"RFE - n_features_to_select={n_features}:")
    print(f"Training R² Score: {train_r2_score_rfe:.4f}, Validation R² Score: {val_r2_score_rfe:.4f}, Features Remaining: {X_train_rfe.shape[1]}\n")

RFE - n_features_to_select=5:
Training R² Score: 0.7281, Validation R² Score: 0.6817, Features Remaining: 5

RFE - n_features_to_select=10:
Training R² Score: 0.7896, Validation R² Score: 0.6444, Features Remaining: 10

RFE - n_features_to_select=15:
Training R² Score: 0.7990, Validation R² Score: 0.7377, Features Remaining: 15

RFE - n_features_to_select=20:
Training R² Score: 0.8004, Validation R² Score: 0.7468, Features Remaining: 17

RFE - n_features_to_select=25:
Training R² Score: 0.8004, Validation R² Score: 0.7468, Features Remaining: 17



Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [16]:
# Your code here
alpha_values = [0.01, 0.1, 1, 10, 100]  # Adjust these values based on your dataset
for alpha in alpha_values:
    lasso = LassoCV(alphas=[alpha], cv=5)
    lasso.fit(X_train, y_train)
    
    # Get selected features
    selected_features_lasso = np.where(lasso.coef_ != 0)[0]  # Indices of selected features
    X_train_lasso = X_train.iloc[:, selected_features_lasso]
    X_val_lasso = X_val.iloc[:, selected_features_lasso]
    
    # Fit the model and calculate R² scores
    model_rfe.fit(X_train_lasso, y_train)
    train_r2_score_lasso = model_rfe.score(X_train_lasso, y_train)
    val_r2_score_lasso = model_rfe.score(X_val_lasso, y_val)
    
    print(f"Lasso - alpha={alpha}:")
    print(f"Training R² Score: {train_r2_score_lasso:.4f}, Validation R² Score: {val_r2_score_lasso:.4f}, Features Remaining: {len(selected_features_lasso)}\n")

Lasso - alpha=0.01:
Training R² Score: 0.8004, Validation R² Score: 0.7468, Features Remaining: 17

Lasso - alpha=0.1:
Training R² Score: 0.8004, Validation R² Score: 0.7468, Features Remaining: 17

Lasso - alpha=1:
Training R² Score: 0.8004, Validation R² Score: 0.7468, Features Remaining: 17

Lasso - alpha=10:
Training R² Score: 0.8004, Validation R² Score: 0.7468, Features Remaining: 17

Lasso - alpha=100:
Training R² Score: 0.8004, Validation R² Score: 0.7468, Features Remaining: 17



Compare the results. Which features would you choose to use?

# Your written answer here
RFE Results: You will observe how the training and validation ( R^2 ) scores change as you vary the number of features. If you notice that the validation score starts to decrease as you increase the number of features, it may indicate overfitting.

Lasso Results: The Lasso regression results will show how different alpha values affect the number of selected features and the model's performance. A higher alpha typically results in fewer features being

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [22]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import pandas as pd

# Assuming X_train, X_val, and X_test have already been defined
# Combine train and validation sets for scaling and polynomial feature generation
X_combined = pd.concat([X_train, X_val])  # Combine train and validation sets

# Step 1: Scale the combined data
scaler = StandardScaler()
X_combined_scaled = scaler.fit_transform(X_combined)

# Step 2: Add polynomial features to the combined scaled data
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_combined_poly = poly.fit_transform(X_combined_scaled)

# Step 3: Prepare X_test
# Fill missing columns in X_test with zeros or NaNs
for col in X_combined.columns:
    if col not in X_test.columns:
        X_test[col] = 0  # or use NaN: X_test[col] = np.nan

# Now you can proceed to scale and transform
X_test_scaled = scaler.transform(X_test)  # Scale X_test
X_test_poly = poly.transform(X_test_scaled)  # Generate polynomial features for X_test

# Now X_test_poly is ready for model evaluation

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [23]:
# Your code here
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Combine training and validation sets
X_train_val = pd.concat([X_train, X_val])
y_train_val = pd.concat([y_train, y_val])

# Scale and create polynomial features for the combined training and validation sets
X_train_val_scaled = scaler.transform(X_train_val)
X_train_val_poly = poly.transform(X_train_val_scaled)

# Fit Lasso model on the combined training and validation sets
lasso = LassoCV(cv=5)
lasso.fit(X_train_val_poly, y_train_val)

# Predict on the test set
y_pred = lasso.predict(X_test_poly)

# Calculate R-Squared and MSE
r_squared = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"Test R² Score: {r_squared:.4f}")
print(f"Test MSE: {mse:.4f}")

Test R² Score: 0.7536
Test MSE: 1726415076.4056


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.